In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from churn_predictor.dataset import get_cleaned_dataframe
import churn_predictor.plot as plots

In [4]:
df = get_cleaned_dataframe()

In [5]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,15574012,Chu,645,Spain,Male,44.0,8,113755.78,2,1.0,0.0,149756.71,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,15584532,Liu,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9994,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9995,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9996,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0
